# Création de prompts pour le nommage et la description des clusters

In [16]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"

In [17]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_hommes     <- mean(cluster_data$ses_genderMale, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$ses_immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$ses_languageEnglish, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$ses_languageFrench, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethnicityWhite, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_sexOrientationHetero, na.rm = TRUE) * 100
  educ_mean_BHS       <- mean(cluster_data$ses_educBHS, na.rm = TRUE) * 100
  educ_mean_PostHS       <- mean(cluster_data$ses_educPostHS, na.rm = TRUE) * 100
  educ_mean_Univ       <- mean(cluster_data$ses_educUniv, na.rm = TRUE) * 100
  age_mean        <- mean(cluster_data$ses_age, na.rm = TRUE) * 80
  revenu_moyen_Low    <- mean(cluster_data$ses_incomeLow, na.rm = TRUE) * 100
  revenu_moyen_Mid    <- mean(cluster_data$ses_incomeMid, na.rm = TRUE) * 100
  revenu_moyen_High    <- mean(cluster_data$ses_incomeHigh, na.rm = TRUE) * 100

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
  "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona ", cluster_value, " :\n\n",
  
  "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "green")) {
    paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
  } else {
    "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
  },
  
  "\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "red")) {
    paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
  } else {
    "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
  },
  
  "\n\nCe cluster est composé de :\n", 
  "Hommes: ", round(perc_hommes, 1), "%, ",
  "Immigrants: ", round(perc_immigrants, 1), "%\n",
  "Âge: ", round(age_mean, 1), " ans\n",
  "Revenus:\n",
  "  Revenu Bas: ", round(revenu_moyen_Low, 1), " %\n",
  "  Revenu Mid: ", round(revenu_moyen_Mid, 1), " %\n",
  "  Revenu High: ", round(revenu_moyen_High, 1), " %\n",
  "Langues:\n",
  "  En: ", round(perc_langEn, 1), "%, ",
  "Fr: ", round(perc_langFr, 1), "%, ",
  "Autres: ", round(perc_ses_languageOther, 1), "%\n",
  "EducBHS: ", round(educ_mean_BHS, 1), "%\n",
  "EducPostHS: ", round(educ_mean_PostHS, 1), "%\n",
  "EducUniv: ", round(educ_mean_Univ, 1), "%\n",
  "Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
  "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
))

  prompts[[cluster_value]] <- prompt_description
}


In [18]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_exerciseYoga\", \"ses_dwellingApp\", \"ses_immigrant\", \"ses_incomeHigh\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_exerciseGym\", \"lifestyle_smokeFreq\")\n\nCe cluster est composé de :\nHommes: 27.3%, Immigrants: 40.9%\nÂge: 40 ans\nRevenus:\n  Revenu Bas: 4.5 %\n  Revenu Mid: 59.1 %\n  Revenu High: 36.4 %\nLangues:\n  En: 72.7%, Fr: 27.3%, Autres: 0%\nEducBHS: 13.6%\nEducPostHS: 22.7%\nEducUniv: 63.6%\nEthnie (Blanc): 63.6%\nOrientation (Hétéro): 86.4%\n"

$`10`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 10 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_clothingStyleOther\", \"lifestyle_hasTattoos\", \"ses_incomeLow\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"ses_educUniv\", \"ses_immigrant\")\n\nCe cluster est composé de :\nHommes: 43.9%, Immigrants: 0%\nÂge: 45.9 ans\nRevenus:\n  Revenu Bas: 24.4 %\n  Revenu Mid: 65.9 %\n  Revenu High: 9.8 %\nLangues:\n  En: 58.5%, Fr: 39%, Autres: 2.4%\nEducBHS: 31.7%\nEducPostHS: 39%\nEducUniv: 29.3%\nEthnie (Blanc): 82.9%\nOrientation (Hétéro): 80.5%\n"

$`11`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 11 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_favAlcoolBeer\", \"lifestyle_smokeFreq\", \"lifestyle_typeTransportCar\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_exerciseGym\", \"lifestyle_favAlcoolDontDrink\", \"lifestyle_favAlcoolRedWine\", \"lifestyle_favAlcoolWhiteWine\", \"lifestyle_hasTattoos\", \"lifestyle_typeTransportNoCar\", \"ses_incomeLow\", \"ses_languageEnglish\")\n\nCe cluster est composé de :\nHommes: 86.8%, Immigrants: 18.7%\nÂge: 51.4 ans\nRevenus:\n  Revenu Bas: 3.3 %\n  Revenu Mid: 82.4 %\n  Revenu High: 14.3 %\nLangues:\n  En: 50.5%, Fr: 42.9%, Autres: 6.6%\nEducBHS: 22%\nEducPostHS: 40.7%\nEducUniv: 37.4%\nEthnie (Blanc): 84.6%\nOrientation (Hétéro): 90.1%\n"

$`2`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_consClothesFrip\", \"lifestyle_eatMeatFreq\", \"lifestyle_favAlcoolDontDrink\", \"lifestyle_ownPetCat\", \"lifestyle_typeTransportCar\", \"ses_age\", \"ses_languageEnglish\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_consCoffeeTimHortons\", \"lifestyle_favAlcoolBeer\", \"lifestyle_ownPetNone\", \"lifestyle_typeTransportNoCar\", \"ses_educUniv\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 41.5%, Immigrants: 9.8%\nÂge: 52.7 ans\nRevenus:\n  Revenu Bas: 16.3 %\n  Revenu Mid: 74.8 %\n  Revenu High: 8.9 %\nLangues:\n  En: 99.2%, Fr: 0%, Autres: 0.8%\nEducBHS: 29.3%\nEducPostHS: 42.3%\nEducUniv: 28.5%\nEthnie (Blanc): 90.2%\nOrientation (Hétéro): 90.2%\n"

$`3`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 3 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_consCoffeeNone\", \"lifestyle_exerciseNone\", \"lifestyle_favAlcoolWhiteWine\", \"lifestyle_smokeFreq\", \"ses_age\", \"ses_educBHS\")\n\nVariables avec une moyenne significativement plus basse que la moyenne d

In [19]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [20]:
reponses_openai

$`1`
[1] "Prénom proposé : Sophie\n\nDescription du persona 1 :\n\nSophie est une femme de 40 ans vivant au Canada. Elle est immigrante, ce qui se reflète dans la diversité culturelle qu'elle apporte à son environnement. Pratiquant régulièrement le yoga, elle préfère cette activité au gymnase pour rester en forme et se relaxer. Sa santé est importante pour elle, et elle ne fume pas.\n\nElle vit dans une habitation moderne et apprécie le confort que cela lui procure. Avec un revenu élevé, Sophie a su construire une carrière réussie et épanouissante. Elle est bien éduquée, ayant un diplôme universitaire qui lui permet de travailler dans un domaine qu'elle aime. Bien qu'elle parle principalement anglais, elle connaît aussi le français, ce qui lui donne un avantage dans un pays bilingue comme le Canada.\n\nD'origine majoritairement blanche, Sophie s'intègre bien dans la société canadienne tout en apportant sa propre perspective en tant qu'immigrante. Elle vit une vie conventionnelle avec une orientation hétérosexuelle et apprécie les valeurs familiales et communautaires."

$`10`
[1] "Pour une personne habitant au Canada correspondant au persona 10, un prénom qui pourrait convenir est \"Marc\". Voici une brève description de ce persona :\n\n---\n\n**Nom** : Marc\n\n**Âge** : 45 ans\n\n**Genre** : Homme\n\n**Langues parlées** : Marc est principalement anglophone, mais il parle également français.\n\n**Origines** : Bien qu'il soit descendant de familles établies au Canada depuis plusieurs générations, Marc s'identifie fortement à la culture canadienne-française.\n\n**Situation socio-économique** : Marc a un revenu moyen, ce qui lui permet de vivre confortablement sans extravagances. Cependant, il fait partie de ceux dont le revenu est considéré comme bas au regard des standards nationaux.\n\n**Éducation** : Marc a complété des études postsecondaires, mais n'a pas poursuivi jusqu'à l'université, ce qui est moins fréquent dans la population générale.\n\n**Style de vie** : Marc adopte un style vestimentaire unique et n'hésite pas à porter des vêtements en dehors des tendances conventionnelles. Il se distingue également par ses nombreux tatouages, qui reflètent souvent des aspects importants de sa vie et de sa personnalité.\n\n**Profil culturel** : Bien qu'il ait grandi dans une communauté blanche et hétérosexuelle, Marc apprécie la diversité culturelle et est ouvert d'esprit. Il n'est pas immigré et n'a pas d'ascendance immigrée directe.\n\n**Valeurs et loisirs** : Marc valorise l'authenticité et l'expression personnelle. Il aime explorer ses intérêts artistiques, souvent en lien avec ses tatouages, et participe à des événements culturels locaux qui célèbrent la diversité et la créativité.\n\n--- \n\nCette description vise à capturer les caractéristiques principales du persona en tenant compte des variables spécifiques fournies."

$`11`
[1] "Pour une personne vivant au Canada, et plus spécifiquement au Québec, un prénom approprié pourrait être \"Marc\".\n\n### Description du persona \"Marc\" :\n\n- **Nom** : Marc\n- **Âge** : 51 ans\n- **Genre** : Homme\n- **Origine** : Principalement d'origine québécoise, avec une probabilité faible d'immigration (18.7%)\n- **Langue principale** : Français, bien qu'il soit aussi capable de parler anglais\n- **Région** : Province de Québec\n- **Statut socio-économique** : Revenu moyen, avec une proportion significative ayant un revenu élevé\n- **Éducation** : A probablement fait des études postsecondaires, avec une bonne chance d'avoir un diplôme universitaire\n- **Profession** : Peut exercer une profession nécessitant un niveau d'éducation élevé\n- **Mode de vie** :\n  - Préfère la bière comme boisson alcoolisée\n  - Fume plus que la moyenne de la population\n  - Utilise principalement la voiture pour se déplacer\n  - Fait peu d’exercice au gym\n  - N’a probablement pas de tatouages\n- **Culture et appartenance ethnique** : Majoritairement de descendance blanche\n- **Orientation sexuelle** : Hétérosexu

In [25]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [26]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""lifestyle_exerciseYoga"", ""ses_dwellingApp"", ""ses_immigrant"", ""ses_incomeHigh"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""lifestyle_exerciseGym"", ""lifestyle_smokeFreq"") Ce cluster est composé de : Hommes: 27.3%, Immigrants: 40.9% Âge: 40 ans Revenus: Revenu Bas: 4.5 % Revenu Mid: 59.1 % Revenu High: 36.4 % Langues: En: 72.7%, Fr: 27.3%, Autres: 0% EducBHS: 13.6% EducPostHS: 22.7% EducUniv: 63.6% Ethnie (Blanc): 63.6% Orientation (Hétéro): 86.4%","Prénom proposé : Sophie Description du persona 1 : Sophie est une femme de 40 ans vivant au Canada. Elle est immigrante, ce qui se reflète dans la diversité culturelle qu'elle apporte à son environnement. Pratiquant régulièrement le yoga, elle préfère cette activité au gymnase pour rester en forme et se relaxer. Sa santé est importante pour elle, et elle ne fume pas. Elle vit dans une habitation moderne et apprécie le confort que cela lui procure. Avec un revenu élevé, Sophie a su construire une carrière réussie et épanouissante. Elle est bien éduquée, ayant un diplôme universitaire qui lui permet de travailler dans un domaine qu'elle aime. Bien qu'elle parle principalement anglais, elle connaît aussi le français, ce qui lui donne un avantage dans un pays bilingue comme le Canada. D'origine majoritairement blanche, Sophie s'intègre bien dans la société canadienne tout en apportant sa propre perspective en tant qu'immigrante. Elle vit une vie conventionnelle avec une orientation hétérosexuelle et apprécie les valeurs familiales et communautaires."
10,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 10 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""lifestyle_clothingStyleOther"", ""lifestyle_hasTattoos"", ""ses_incomeLow"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""ses_educUniv"", ""ses_immigrant"") Ce cluster est composé de : Hommes: 43.9%, Immigrants: 0% Âge: 45.9 ans Revenus: Revenu Bas: 24.4 % Revenu Mid: 65.9 % Revenu High: 9.8 % Langues: En: 58.5%, Fr: 39%, Autres: 2.4% EducBHS: 31.7% EducPostHS: 39% EducUniv: 29.3% Ethnie (Blanc): 82.9% Orientation (Hétéro): 80.5%","Pour une personne habitant au Canada correspondant au persona 10, un prénom qui pourrait convenir est ""Marc"". Voici une brève description de ce persona : --- **Nom** : Marc **Âge** : 45 ans **Genre** : Homme **Langues parlées** : Marc est principalement anglophone, mais il parle également français. **Origines** : Bien qu'il soit descendant de familles établies au Canada depuis plusieurs générations, Marc s'identifie fortement à la culture canadienne-française. **Situation socio-économique** : Marc a un revenu moyen, ce qui lui permet de vivre confortablement sans extravagances. Cependant, il fait partie de ceux dont le revenu est considéré comme bas au regard des standards nationaux. **Éducation** : Marc a complété des études postsecondaires, mais n'a pas poursuivi jusqu'à l'université, ce qui est moins fréquent dans la population générale. **Style de vie** : Marc adopte un style vestimentaire unique et n'hésite pas à porter des vêtements en dehors des tendances conventionnelles. Il se distingue également par ses nombreux tatouages, qui reflètent souvent des aspects importants de sa vie et de sa personnalité. **Profil culturel** : Bien qu'il ait grandi dans une communauté blanche et hétérosexuelle, Marc apprécie la diversité culturelle et est ouvert d'esprit. Il n'est pas immigré et n'a pas d'ascendance immigrée directe. **Valeurs et loisirs** : Marc valorise l'authenticité et l'expression personnelle. Il aime explorer ses inté

In [27]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [28]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")